<a href="https://colab.research.google.com/github/LH-1169213/Equipo3/blob/main/SVD_Equipo09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Samara García González A01273001
*   Laura Elena Hernández Mata A01169213
*   Evelyn Aylin Rendon Medina A01748750
*   Emmanuel González Calitl A01320739
*   Carlos Enrique Diaz Ayala A01793816


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [1]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

#**Ejercicio-1.** 

In [2]:
#rating_final.csv
df_ratingfinal = pd.read_csv("https://raw.githubusercontent.com/LH-1169213/Equipo3/main/rating_final.csv")
df_ratingfinal.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [3]:
#Obteniendo datos de valor del archivo rating_final.csv
UserID = np.array(df_ratingfinal[['userID']]).flatten()
print(UserID.shape)
rating = np.array(df_ratingfinal[['rating']]).flatten()
print(rating.shape)

(1161,)
(1161,)


In [4]:
# u'\xe9'.encode('latin-1')

#geoplaces2.csv
df_geoplaces = pd.read_csv("https://raw.githubusercontent.com/LH-1169213/Equipo3/main/geoplaces2.csv", encoding = "latin-1")
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [5]:
# Creamos un diccionario con llave (placeID) y sus nombres
place_map = pd.Series(df_geoplaces.name.values, index = df_geoplaces.placeID).to_dict()
place_map

{134999: 'Kiku Cuernavaca',
 132825: 'puesto de tacos',
 135106: 'El Rincón de San Francisco',
 132667: 'little pizza Emilio Portes Gil',
 132613: 'carnitas_mata',
 135040: 'Restaurant los Compadres',
 132732: 'Taqueria EL amigo ',
 132875: 'shi ro ie',
 132609: 'Pollo_Frito_Buenos_Aires',
 135082: 'la Estrella de Dimas',
 135070: 'Restaurante 75',
 135069: 'Abondance Restaurante Bar',
 135065: 'El angel Restaurante',
 135076: 'Restaurante Pueblo Bonito',
 135086: 'Mcdonalds Parque Tangamanga',
 132870: 'Tortas y hamburguesas el gordo',
 132854: 'Sirlone',
 132937: 'rockabilly ',
 132856: 'Unicols Pizza',
 132668: 'TACOS EL GUERO',
 135033: 'Restaurant El Muladar de Calzada',
 132862: 'La Posada del Virrey',
 134983: 'Restaurant and Bar and Clothesline Carlos N Charlies',
 132851: 'KFC',
 135013: 'Giovannis',
 135042: 'Restaurant Oriental Express',
 132768: 'Mariscos Tia Licha',
 132561: 'cafe ambar',
 135045: 'Restaurante la Gran Via',
 132847: 'don burguers',
 135054: 'Restaurante y 

In [6]:
# Creamos un DF con los valores previamente obtenidos de las dos BD

df = pd.DataFrame({"UserID": UserID,"rating": rating})
df.head()
df.shape


(1161, 2)

In [7]:
# Agregamos el nombre del lugar que están calificando los usuarios

df["place"] = df_ratingfinal['placeID'].map(place_map)
df.head()

,UserID,rating,place
0,U1077,2,Tortas Locas Hipocampo
1,U1077,2,Restaurant la Chalita
2,U1077,2,puesto de tacos
3,U1077,1,Restaurante Marisco Sam
4,U1068,1,vips


In [8]:
#Matriz de utilidad
matriz = df.pivot_table(values='rating', index='UserID', columns='place', fill_value=0)
print('Tamaño de la matriz de utilidad:', matriz.shape)
print('número de filas:',matriz.shape[0])
print('número de columnas:',matriz.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 138
número de columnas: 129


In [9]:
matriz.head(10)

place,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
UserID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
print('Total de elementos de la matriz de utilidad: %d' % (matriz.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(matriz)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(matriz) / matriz.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Porcentaje de elementos diferentes de cero: 5.1%


#**Ejercicio-2.**

In [11]:
#Transpuesta de la matriz de utilidad
#Para obtener la información latente en relación a los lugares de comida
matrizT = matriz.T
print('Tamaño de la matriz de utilidad:', matriz.shape)
print('número de filas:',matrizT.shape[0])
print('número de columnas:',matrizT.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 129
número de columnas: 138


In [12]:
matrizT.head()

UserID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
place,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#El número de componentes debe ser menor que el número de características
SVD = TruncatedSVD(n_components=137, random_state=1)  
SVD.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.5%


#**Ejercicio-3.**

In [44]:
# ENTRENAMOS AL MODELO
New_matrix = SVD.fit_transform(matrizT)
corrMx = np.corrcoef(New_matrix)
print("Shape:")
print(corrMx.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_ej3 = "Subway"

names = matriz.columns
names_list = list(names)
id_restau = names_list.index(sugerencia_ej3)

print("\nID restaurante:")
print(id_restau)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej3 = np.sort(corrMx[id_restau])[::-1][0:12]
resultado_ej3 = recomienda_ej3[-1]
listado_ej3 = list(names[(corrMx[id_restau] > resultado_ej3) & (corrMx[id_restau] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_ej3[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej3)):
  print(listado_ej3[i])

Shape:
(129, 129)

ID restaurante:
88

Correlaciones de los restaurantes
[0.60952087 0.50627768 0.42982383 0.42546717 0.4172185  0.388178
 0.30109049 0.30100248 0.28514922 0.2346343 ]

Listado de recomendaciones:
Chilis Cuernavaca
Log Yin
Mariscos Tia Licha
Mikasa
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Restaurant los Pinos
Rincon del Bife
Sanborns Casa Piedra
Vips


#**Ejercicio-4.**

In [45]:
# REALIZAMOS EL EJERCICIO CON LA REDUCCIÓN DE COMPONENTES
SVD_ej4 = TruncatedSVD(n_components=60, random_state=1)  
SVD_ej4.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_ej4.singular_values_[0:num_sv]).sum() / (SVD_ej4.singular_values_).sum())))

# ENTRENAMOS AL MODELO
Matrix_ej4 = SVD_ej4.fit_transform(matrizT)
corrMx_ej4 = np.corrcoef(Matrix_ej4)
corrMx_ej4.shape

#OBTENEMOS EL ID DEL RESTAURANTE
sugerencia_ej4 = "Subway"

names_ej4 = matriz.columns
names_list_ej4 = list(names_ej4)
id_restau_4 = names_list_ej4.index(sugerencia_ej4)

print("\nID restaurante:")
print(id_restau_4)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej4 = np.sort(corrMx_ej4[id_restau_4])[::-1][0:12]
resultado_ej4 = recomienda_ej4[-1]
listado_ej4 = list(names[(corrMx_ej4[id_restau_4] > resultado_ej4) & (corrMx_ej4[id_restau_4] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_ej4[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej4)):
  print(listado_ej4[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
81.8%

ID restaurante:
88

Correlaciones de los restaurantes
[0.77764158 0.6961823  0.59739657 0.53014557 0.50026555 0.45014639
 0.42176904 0.34274103 0.2970983  0.24040946]

Listado de recomendaciones:
Chilis Cuernavaca
Mariscos Tia Licha
Mikasa
Paniroles
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Restaurant los Pinos
Rincon del Bife
Sanborns Casa Piedra
Vips


#**Ejercicio-5.**

In [46]:
# CALCULAMOS CON LA TOTALIDAD DE LA INFORMACIÓN

SVD_ej5 = TruncatedSVD(n_components=137, random_state=1)  
SVD_ej5.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_ej5.singular_values_[0:num_sv]).sum() / (SVD_ej5.singular_values_).sum())))

Matrix_ej5 = SVD_ej5.fit_transform(matrizT)
corrMx_ej5 = np.corrcoef(Matrix_ej5)
print("\n Shape:")
print(corrMx_ej5.shape)

#OBTENEMOS EL ID DEL RESTAURANTE
sugerencia_ej5 = "tacos de barbacoa enfrente del Tec"

names_ej5 = matriz.columns
names_list_ej5 = list(names_ej5)
id_restau_5 = names_list_ej5.index(sugerencia_ej5)

print("\nID Restaurante:")
print(id_restau_5)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej5 = np.sort(corrMx_ej5[id_restau_5])[::-1][0:12]
resultado_ej5 = recomienda_ej5[-1]
listado_ej5 = list(names[(corrMx_ej5[id_restau_5] > resultado_ej5) & (corrMx_ej5[id_restau_5] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_ej5[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej5)):
  print(listado_ej5[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.5%

 Shape:
(129, 129)

ID Restaurante:
124

Correlaciones de los restaurantes
[0.81718898 0.66530537 0.65384927 0.43718421 0.41276452 0.40779492
 0.25847939 0.19517409 0.19112045 0.17093947]

Listado de recomendaciones:
Carreton de Flautas y Migadas
Hamburguesas La perica
Little Cesarz
Taqueria EL amigo 
carnitas mata calle Emilio Portes Gil
little pizza Emilio Portes Gil
palomo tec
puesto de gorditas
tacos abi
vips


In [48]:
# CALCULAMOS CON UNA PARTE DE LA INFORMACIÓN
SVD_ej5_V2 = TruncatedSVD(n_components=80, random_state=1)  
SVD_ej5_V2.fit(matrizT)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_ej5_V2.singular_values_[0:num_sv]).sum() / (SVD_ej5_V2.singular_values_).sum())))

# ENTRENAMOS AL MODELO
Matrix_ej5_V2 = SVD_ej5_V2.fit_transform(matrizT)
corrMx_ej5_V2 = np.corrcoef(Matrix_ej5_V2)
print("\n Shape:")
print(corrMx_ej5_V2.shape)

#OBTENEMOS EL ID DEL RESTAURANTE
sugerencia_ej5_V2 = "tacos de barbacoa enfrente del Tec"

names_ej5_V2 = matriz.columns
names_list_ej5_V2 = list(names_ej5_V2)
id_restau_5_V2 = names_list_ej5_V2.index(sugerencia_ej5_V2)

print("\n ID Restaurante:")
print(id_restau_5_V2)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_ej5_V2 = np.sort(corrMx[id_restau_5_V2])[::-1][0:12]
resultado_ej5_V2 = recomienda_ej5_V2[-1]
listado_ej5_V2 = list(names[(corrMx_ej5_V2[id_restau_5_V2] > resultado_ej5_V2) & (corrMx_ej5_V2[id_restau_5_V2] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_ej5_V2[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_ej5_V2)):
  print(listado_ej5_V2[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
84.0%

 Shape:
(129, 129)

 ID Restaurante:
124

Correlaciones de los restaurantes
[0.81718898 0.66530537 0.65384927 0.43718421 0.41276452 0.40779492
 0.25847939 0.19517409 0.19112045 0.17093947]

Listado de recomendaciones:
Carreton de Flautas y Migadas
Gorditas Dona Tota
Hamburguesas La perica
Little Cesarz
Taqueria EL amigo 
carnitas mata calle Emilio Portes Gil
carnitas_mata
little pizza Emilio Portes Gil
puesto de gorditas
tacos abi
vips


# **Conclusiones ejercicio 5**


El coeficiente de correlación de Pearson nos permite conocer la relación entre dos variables continuas, toma valores entre +1 y -1, mientras que si la relación es 0 indica que las variables no estan relacionadas, si el valor es positivo indica que ambas  variables están relacionadas y que si el valor de una crece, también lo hará la otra variable.

En este ejercicio se obtienen las 10 mejores recomendaciones usando la correlación de Pearson con base en la información de alguien que evaluó bien al restaurante tacos de barbacoa enfrente del Tec, como resultado se obtuvieron las siguientes recomendaciones: 

Listado de recomendaciones con la reducción de componentes:
#####*Carreton de Flautas y Migadas
#####*Hamburguesas La perica
#####*Little Cesarz
#####*Taqueria EL amigo 
#####*carnitas mata calle Emilio Portes Gil
#####*little pizza Emilio Portes Gil
#####*palomo tec
#####*puesto de gorditas
#####*tacos abi
#####*vips
 
Como se puede ver al obtener los porcentajes , el restaurante con mayor correlación es el de Carreton de Flautas y Migadas  con 86.55982233473783 % y el de menor correlación es vips con un 17.796731760012992 %

Cabe mencionar que los resultados varían con respecto al ejercicio 3 y 4, ya que en este caso el restaurante base fue "tacos de barbacoa enfrente del Tec", mientras que en los otros dos ejercicios había sido "subway".

Igualmente, la diferencia al reducir las dimensiones sólo fue el restaurante "Carnitas Mata" que aparece con los 137 compontentes; y se integró "palomo Tec" en la versión de 80 componentes. 

#**Ejercicio-6.**

In [51]:
#Obteniendo datos de valor del archivo rating_final.csv
UserID = np.array(df_ratingfinal[['userID']]).flatten()
print(UserID.shape)
food_rating = np.array(df_ratingfinal[['food_rating']]).flatten()
print(food_rating.shape)

(1161,)
(1161,)


In [52]:
# Creamos un DF con los valores previamente obtenidos de las dos BD

df_food = pd.DataFrame({"UserID": UserID,"food_rating": food_rating})
df_food.head()
df_food.shape

(1161, 2)

In [53]:
# Agregamos el nombre del lugar que están calificando los usuarios

df_food["place"] = df_ratingfinal['placeID'].map(place_map)
df_food.head()

,UserID,food_rating,place
0,U1077,2,Tortas Locas Hipocampo
1,U1077,2,Restaurant la Chalita
2,U1077,2,puesto de tacos
3,U1077,2,Restaurante Marisco Sam
4,U1068,1,vips


In [54]:
#Matriz de utilidad
matriz_food = df_food.pivot_table(values='food_rating', index='UserID', columns='place', fill_value=0)
print('Tamaño de la matriz de utilidad:', matriz_food.shape)
print('número de filas:',matriz_food.shape[0])
print('número de columnas:',matriz_food.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 138
número de columnas: 129


In [55]:
matriz_food.head(10)

place,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
UserID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [56]:
print('Total de elementos de la matriz de utilidad: %d' % (matriz_food.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(matriz_food)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(matriz_food) / matriz_food.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 895
Porcentaje de elementos diferentes de cero: 5.0%


In [57]:
#Transpuesta de la matriz de utilidad
#Para obtener la información latente en relación a los lugares de comida
matrizT_food = matriz_food.T
print('Tamaño de la matriz de utilidad:', matriz_food.shape)
print('número de filas:',matrizT_food.shape[0])
print('número de columnas:',matrizT_food.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 129
número de columnas: 138


In [58]:
matrizT_food.head()

UserID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
place,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# REALIZAMOS EL EJERCICIO CON LA TOTALIDAD DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_food = TruncatedSVD(n_components=137, random_state=1)  
SVD_food.fit(matrizT_food)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_food.singular_values_[0:num_sv]).sum() / (SVD_food.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_food = SVD_food.fit_transform(matrizT_food)
corrMx_food = np.corrcoef(New_matrix_food)
print("\nShape:")
print(corrMx_food.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_food = "Subway"

names_food = matriz_food.columns
names_list_food = list(names_food)
id_restau_food = names_list.index(sugerencia_food)

print("\nID restaurante:")
print(id_restau_food)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_food = np.sort(corrMx_food[id_restau_food])[::-1][0:12]
resultado_food = recomienda_food[-1]
listado_food = list(names_food[(corrMx_food[id_restau_food] > resultado_food) & (corrMx_food[id_restau_food] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_food[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_food)):
  print(listado_food[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.9%

Shape:
(129, 129)

ID restaurante:
88

Correlaciones de los restaurantes
[0.81641337 0.46894969 0.4421477  0.41522205 0.38181946 0.36565669
 0.33170678 0.31208236 0.2503574  0.23270837]

Listado de recomendaciones:
Kiku Cuernavaca
Mariscos Tia Licha
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Sanborns Casa Piedra
Vips


In [60]:
# REALIZAMOS EL EJERCICIO CON LA REDUCCIÓN DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_food_v2 = TruncatedSVD(n_components=75, random_state=1)  
SVD_food_v2.fit(matrizT_food)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_food_v2.singular_values_[0:num_sv]).sum() / (SVD_food_v2.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_food_v2 = SVD_food_v2.fit_transform(matrizT_food)
corrMx_food_v2 = np.corrcoef(New_matrix_food_v2)
print("\n Shape:")
print(corrMx_food_v2.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_food_v2 = "Subway"

names_food_v2 = matriz_food.columns
names_list_food_v2 = list(names_food_v2)
id_restau_food_v2 = names_list.index(sugerencia_food_v2)

print("\n ID restaurante:")
print(id_restau_food_v2)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_food_v2 = np.sort(corrMx_food_v2[id_restau_food_v2])[::-1][0:12]
resultado_food_v2 = recomienda_food_v2[-1]
listado_food_v2 = list(names_food_v2[(corrMx_food_v2[id_restau_food_v2] > resultado_food_v2) & (corrMx_food_v2[id_restau_food_v2] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_food_v2[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_food_v2)):
  print(listado_food_v2[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
84.0%

 Shape:
(129, 129)

 ID restaurante:
88

Correlaciones de los restaurantes
[0.87063469 0.49676023 0.45979207 0.44657809 0.43323238 0.38546085
 0.33467078 0.31837836 0.25267134 0.25088067]

Listado de recomendaciones:
Kiku Cuernavaca
Mariscos Tia Licha
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Sanborns Casa Piedra
Vips


# **Conclusiones ejercicio 6**
En este ejercicio se genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada a la comida (food_rating). Obteniendo los siguientes resultados: 
Tamaño de la matriz de utilidad: (138, 129)
número de filas: 138
número de columnas: 129, la cantidad de información simplificada con los 5 vectores singulares obtenidos fue de 85.9% y el listado de recomendaciones fué el siguiente:
Listado de recomendaciones:
####*Kiku Cuernavaca
####*Mariscos Tia Licha
####*Paniroles
####*Restaurant Familiar El Chino
####*Restaurant Las Mananitas
####*Restaurant Teely
####*Restaurant and Bar and Clothesline Carlos N Charlies
####*Rincon del Bife
####*Sanborns Casa Piedra
####*Vips

Siendo KikU Cuernavaca el de mayor correlación con 87.063% y el de menor correlación fupe Vips con 0.088%

En este caso, a pesar de haber reducido los componentes de 137 en la primera ejecución a 75, se aprecia que el resultado es el mismo en cuanto a las recomendaciones. 

#**Ejercicio-7.**

In [62]:
#Obteniendo datos de valor del archivo rating_final.csv
UserID = np.array(df_ratingfinal[['userID']]).flatten()
print(UserID.shape)
service_rating = np.array(df_ratingfinal[['service_rating']]).flatten()
print(service_rating.shape)

(1161,)
(1161,)


In [63]:
# Creamos un DF con los valores previamente obtenidos de las dos BD

df_service = pd.DataFrame({"UserID": UserID,"service_rating": service_rating})
df_service.head()
df_service.shape

(1161, 2)

In [64]:
# Agregamos el nombre del lugar que están calificando los usuarios

df_service["place"] = df_ratingfinal['placeID'].map(place_map)
df_service.head()

,UserID,service_rating,place
0,U1077,2,Tortas Locas Hipocampo
1,U1077,1,Restaurant la Chalita
2,U1077,2,puesto de tacos
3,U1077,2,Restaurante Marisco Sam
4,U1068,2,vips


In [65]:
#Matriz de utilidad
matriz_service = df_service.pivot_table(values='service_rating', index='UserID', columns='place', fill_value=0)
print('Tamaño de la matriz de utilidad:', matriz_service.shape)
print('número de filas:',matriz_service.shape[0])
print('número de columnas:',matriz_service.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 138
número de columnas: 129


In [66]:
matriz_service.head(10)

place,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
UserID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
print('Total de elementos de la matriz de utilidad: %d' % (matriz_service.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(matriz_service)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(matriz_service) / matriz_service.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 846
Porcentaje de elementos diferentes de cero: 4.8%


In [68]:
#Transpuesta de la matriz de utilidad
#Para obtener la información latente en relación a los lugares de comida
matrizT_service = matriz_service.T
print('Tamaño de la matriz de utilidad:', matriz_service.shape)
print('número de filas:',matrizT_service.shape[0])
print('número de columnas:',matrizT_service.shape[1])

Tamaño de la matriz de utilidad: (138, 129)
número de filas: 129
número de columnas: 138


In [69]:
matrizT_service.head()

UserID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
place,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
# REALIZAMOS EL EJERCICIO CON LA TOTALIDAD DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_service = TruncatedSVD(n_components=137, random_state=1)  
SVD_service.fit(matrizT_service)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_service.singular_values_[0:num_sv]).sum() / (SVD_service.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_service = SVD_service.fit_transform(matrizT_service)
corrMx_service = np.corrcoef(New_matrix_service)
print("\nShape:")
print(corrMx_service.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_service = "Subway"

names_service = matriz_service.columns
names_list_service = list(names_service)
id_restau_service = names_list_service.index(sugerencia_service)

print("\nID restaurante:")
print(id_restau_service)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_service = np.sort(corrMx_service[id_restau_service])[::-1][0:12]
resultado_service = recomienda_service[-1]
listado_service = list(names_service[(corrMx_service[id_restau_service] > resultado_service) & (corrMx_food[id_restau_service] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_service[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_service)):
  print(listado_service[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
85.6%

Shape:
(129, 129)

ID restaurante:
88

Correlaciones de los restaurantes
[0.77297314 0.60016063 0.51873147 0.42119986 0.36593368 0.23069783
 0.22626624 0.19275959 0.18286948 0.17874009]

Listado de recomendaciones:
Chilis Cuernavaca
Giovannis
Kiku Cuernavaca
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Vips


In [74]:
# REALIZAMOS EL EJERCICIO CON LA REDUCCIÓN DE COMPONENTES
# El número de componentes debe ser menor que el número de características
SVD_service_v2 = TruncatedSVD(n_components=60, random_state=1)  
SVD_service_v2.fit(matrizT_service)

num_sv = 5

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD_service_v2.singular_values_[0:num_sv]).sum() / (SVD_service_v2.singular_values_).sum())))

# ENTRENAMOS AL MODELO
New_matrix_service_v2 = SVD_service_v2.fit_transform(matrizT_service)
corrMx_service_v2 = np.corrcoef(New_matrix_service_v2)
print("\nShape:")
print(corrMx_service_v2.shape)

#OBTENEMOS EL ID DEL RESTAURANTE

sugerencia_service_v2 = "Subway"

names_service_v2 = matriz_service.columns
names_list_service_v2 = list(names_service_v2)
id_restau_service_v2 = names_list_service_v2.index(sugerencia_service_v2)

print("\nID restaurante:")
print(id_restau_service_v2)

# OBTENEMOS LAS RECOMENDACIONES ORDENADAS DE MAYOR A MENOR
recomienda_service_v2 = np.sort(corrMx_service_v2[id_restau_service_v2])[::-1][0:12]
resultado_service_v2 = recomienda_service_v2[-1]
listado_service_v2 = list(names_service_v2[(corrMx_service_v2[id_restau_service_v2] > resultado_service_v2) & (corrMx_food_v2[id_restau_service_v2] < 0.99)])[:11]

# IMPRIMIMOS LAS CORRELACIONES OBTENIDAS
print("\nCorrelaciones de los restaurantes")
print(recomienda_service_v2[1:11])

# IMPRIMIMOS LOS RESULTADOS
print("\nListado de recomendaciones:")
for i in range(0, len(listado_service_v2)):
  print(listado_service_v2[i])

Cantidad de información simplificada con los 5 vectores singulares obtenidos:
82.1%

Shape:
(129, 129)

ID restaurante:
88

Correlaciones de los restaurantes
[0.9260933  0.74213457 0.67996005 0.43538637 0.4010418  0.25720902
 0.24805601 0.2251097  0.19310559 0.18678765]

Listado de recomendaciones:
Chilis Cuernavaca
Giovannis
Kiku Cuernavaca
Paniroles
Restaurant Familiar El Chino
Restaurant Las Mananitas
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Vips


# **Conclusiones ejercicio 7**

En este ejercicio se genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada al servicio (service_rating). Obteniendo los siguientes resultados: 
Tamaño de la matriz de utilidad: (138, 129)
número de filas: 129
número de columnas: 138, la cantidad de información simplificada con los 5 vectores singulares obtenidos fue de 85.6% y el listado de recomendaciones fué el siguiente:
Listado de recomendaciones:
####*Chilis Cuernavaca
####*Giovannis
####*Kiku Cuernavaca
####*Paniroles
####*Restaurant Familiar El Chino
####*Restaurant Las Mananitas
####*Restaurant Teely
####*Restaurant and Bar and Clothesline Carlos N Charlies
####*Rincon del Bife
####*Vips

Siendo Chillis Cuernavaca el de mayor correlación con 92.6093299623009 % y el de menor correlación fupe Vips con 18.678764619640205 %

Al igual qu een el caso de food_rating, que fue el ejercicio 6, algo que nos llamó la atención es que aún con la reducción de componentes obtuvimos las mismas recomendaciones con este tercer modelo de service_rating. 

#**Ejercicio-8.**

# **Incluye tus conclusiones finales. En particular indica la coincidencia o no de estos tres modelos.**

Durante el desarrollo de la práctica se trabajó con un sistema de recomendación, donde se emplearon técnicas de reducción de dimensionalidad, en específico se aplicó la descomposición en valores singulares (SVD). Recordemos que trabajar de forma directa con datos que poseen múltiples dimensiones, trae consigo diversas dificultades, tales como la complejidad al momento de analizarlos, interpretarlos, visualizarlos e incluso almacenarlos. De igual forma, suele suceder que los datos que están representados en diversas dimensiones están ‘completos de más’, lo que significa que gran parte de sus dimensiones son redundantes, por lo que podríamos explicar los mismos datos en un número menor, idealmente, conservando toda la información o con una pérdida muy baja (error de reconstrucción pequeño).
Así pues, como sabemos, los sistemas de recomendación se basan en datos relativos a los usuarios y productos/servicios, donde cada usuario x tendrá una tendencia por un artículo y, con una calificación Rxy. Al momento de representar la información anterior, creamos una matriz de utilidad, que es precisamente la que contiene la información requerida para nuestro sistema de recomendación (restaurantes y usuarios, cuyas entradas son calificaciones), de la cual posteriormente se genera su transpuesta debido a que deseamos obtener la información latente con relación a los lugares de comida. Una vez realizado lo anterior, podemos calcular qué tanta información se contiene en la factorización SVD y, basándonos en distintas variables de evaluación para cada modelo (calificación Rxy), desplegar recomendaciones de y para el usuario x. 
Retomando la práctica, en nuestro primer modelo se empleó la variable de evaluación general (rating), en el segundo la particular de la comida (food_rating) y en el tercero, aquella correspondiente a la calidad del servicio (service_rating). Al momento de generar las recomendaciones para cada uno, con su respectivo porcentaje de correlación, pueden observarse grandes similitudes, donde encontramos restaurantes cuya recomendación se comparte por los tres modelos o al menos dos de ello. Son una gran minoría aquellos que no se sugieren por más de uno, lo que nos permite entrever que, tal como indica la teoría, los sistemas de recomendación están cimentados en usuarios, productos/servicios y la tendencia del primero hacia el segundo, determinada por una calificación, cuyo resultado puede no ser muy variable respecto a qué premisa particular se esté evaluando. Consideramos que es relevante resaltar la importancia que tiene el entendimiento en la reducción de componentes para análisis de grandes volúmenes de información, ya que como apreciamos en los modelos de food_rating y service_rating, a pesar de haber reducido los componentes cercano a la mitad, las recomendaciones fueron las mismas que usando todos los componentes. En casos del día a día, consideramos que esto podría facilitar y agilizar la ejecución de búsqueda de patrones o recomendaciones, minimizando así la cantidad de información a utilizar para tener el resultado final. Las variables de entrada, que en este caso es el restaurante para encontrar similitudes, naturalmente, arrojará recomendaciones similares; un ejemplo muy claro fue lo observado entre Subway y Tacos de barbacoa enfrente del Tec.

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**